In [3]:
import numpy as np
import pandas as pd

In [9]:
df = pd.DataFrame({'group': list('aabbabbbabab'),
                    'value': [1, 2, 3, np.nan, 2, 3, np.nan, 1, 7, 3, np.nan, 8]})
print(df)
g1=df.groupby(['group'])['value']
g2=df.fillna(0).groupby(['group'])['value']
#df.fillna（0）命令不会将NaN值替换为0
# .rolling(window, 时间窗口的大小
#          min_periods=None,每个窗口最小观察数量
#          center=False,把窗口的标签设为居中，布尔型
#          win_type=None,窗口的类型
#          on=None,对于df而言，指定要计算滑动窗口的列，值为列名
#          axis=0,对列计算
#          closed=None定义区间的开闭)
s = g2.rolling(3, min_periods=1).sum()    \
    /g1.rolling(3, min_periods=1).count()
#s.reset_index(level=0, drop=True).sort_index()
s

   group  value
0      a    1.0
1      a    2.0
2      b    3.0
3      b    NaN
4      a    2.0
5      b    3.0
6      b    NaN
7      b    1.0
8      a    7.0
9      b    3.0
10     a    NaN
11     b    8.0


group    
a      0     1.000000
       1     1.500000
       4     1.666667
       8     3.666667
       10    4.500000
b      2     3.000000
       3     3.000000
       5     3.000000
       6     3.000000
       7     2.000000
       9     2.000000
       11    4.000000
Name: value, dtype: float64

In [13]:
dti = pd.date_range(start='2015-01-01', end='2015-12-31', freq='B')
s=pd.Series(np.random.rand(len(dti)),index=dti)
s

2015-01-01    0.114082
2015-01-02    0.800342
2015-01-05    0.036465
2015-01-06    0.596256
2015-01-07    0.826098
                ...   
2015-12-25    0.023558
2015-12-28    0.568424
2015-12-29    0.920647
2015-12-30    0.667243
2015-12-31    0.799274
Freq: B, Length: 261, dtype: float64

In [14]:
s[s.index.weekday==2].sum()

25.153048238300865

In [ ]:
s.resample('M').mean()

In [ ]:
s.groupby(pd.Grouper(freq='4M')).idxmax()

In [ ]:
pd.date_range('2015-01-01', '2016-12-31', freq='WOM-3THU')

In [15]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm',
                               'Budapest_PaRis', 'Brussels_londOn'],
              'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
              'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                   'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
                               '12. Air France', '"Swiss Air"']})
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [ ]:
#他们本来应该是每一行增加10，
#填充缺失的数值，并且令数据类型为整数
df['FlightNumber']=df['FlightNumber'].interpolate().astype('int')
# .interpolate(method=‘linear’,
# axis=0,
# limit=None,
# inplace=False,
# limit_direction=‘forward’,
# limit_area=None,
# downcast=None, **kwargs)

# 插值方式
# nearest：最邻近插值法
# zero：阶梯插值
# slinear、linear：线性插值
# quadratic、cubic：2、3阶B样条曲线插值

In [16]:
temp = df.From_To.str.split('_', expand=True)
temp.columns = ['From', 'To']
df = df.join(temp)
df = df.drop('From_To', axis=1)
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045.0,"[23, 47]",KLM(!),LoNDon,paris
1,NaN,[],<Air France> (12),MAdrid,miLAN
2,10065.0,"[24, 43, 87]",(British Airways. ),londON,StockhOlm
3,NaN,[13],12. Air France,Budapest,PaRis
4,10085.0,"[67, 32]","""Swiss Air""",Brussels,londOn


In [17]:
df['From'] = df['From'].str.capitalize()
df['To'] = df['To'].str.capitalize()
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045.0,"[23, 47]",KLM(!),London,Paris
1,NaN,[],<Air France> (12),Madrid,Milan
2,10065.0,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,NaN,[13],12. Air France,Budapest,Paris
4,10085.0,"[67, 32]","""Swiss Air""",Brussels,London


In [18]:
# 需要提取出正确的航司名称。举例：'(British Airways. )'
# 应该改为 'British Airways'.
# str.extract()，可用正则从字符数据中抽取匹配的数据，只返回第一个匹配的数据
df['Airline'] = df['Airline'].str.extract('([a-zA-Z\s]+)',
                                          expand=False).str.strip()
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045.0,"[23, 47]",KLM,London,Paris
1,NaN,[],Air France,Madrid,Milan
2,10065.0,"[24, 43, 87]",British Airways,London,Stockholm
3,NaN,[13],Air France,Budapest,Paris
4,10085.0,"[67, 32]",Swiss Air,Brussels,London
